In [1]:
# importing the libraries
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
import matplotlib.pyplot as plt
from glob import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.model_selection import learning_curve,KFold,cross_val_score,StratifiedKFold
from sklearn.metrics import confusion_matrix

In [2]:
# loading the directories 
training_dir = '../input/cabbagetypescnn/cabbage'

In [3]:
# getting the number of classes i.e. type of fruits
folders = glob(training_dir + '/*')
num_classes = len(folders)
print ('Total Classes = ' + str(num_classes))

Total Classes = 4


In [4]:
# importing the libraries
from keras.models import Model
from keras.layers import Flatten, Dense
from keras.applications.vgg16 import VGG16
#from keras.preprocessing import image

IMAGE_SIZE = [150, 150]  # we will keep the image size as (64,64). You can increase the size for better results. 

# loading the weights of VGG16 without the top layer. These weights are trained on Imagenet dataset.
vgg = VGG16(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)  # input_shape = (64,64,3) as required by VGG

# this will exclude the initial layers from training phase as there are already been trained.
for layer in vgg.layers:
    layer.trainable = False

x = Flatten()(vgg.output)
#x = Dense(128, activation = 'relu')(x)   # we can add a new fully connected layer but it will increase the execution time.
x = Dense(num_classes, activation = 'softmax')(x)  # adding the output layer with softmax function as this is a multi label classification problem.

model = Model(inputs = vgg.input, outputs = x)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



2022-07-19 06:48:05.382541: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-19 06:48:05.550584: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-19 06:48:05.551826: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-19 06:48:05.554149: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

58900480/58889256 [==============================] - 2s 0us/step


In [5]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [6]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    training_dir,
    target_size=IMAGE_SIZE,
    batch_size=16,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    training_dir, # same directory as training data
    target_size=IMAGE_SIZE,
    batch_size=16,
    class_mode='categorical',
    subset='validation') # set as validation data

Found 82 images belonging to 4 classes.
Found 20 images belonging to 4 classes.


In [7]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // 16,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // 16,
    epochs = 8)

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-07-19 06:48:13.778379: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/8


2022-07-19 06:48:15.998856: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


5/5 [==============================] - ETA: 0s - loss: 1.5481 - accuracy: 0.3182

/opt/conda/lib/python3.7/site-packages/PIL/Image.py:993: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


5/5 [==============================] - 13s 919ms/step - loss: 1.5481 - accuracy: 0.3182 - val_loss: 1.0286 - val_accuracy: 0.6875
Epoch 2/8
5/5 [==============================] - 2s 500ms/step - loss: 0.9642 - accuracy: 0.6970 - val_loss: 0.7704 - val_accuracy: 0.7500
Epoch 3/8
5/5 [==============================] - 3s 569ms/step - loss: 0.5967 - accuracy: 0.8636 - val_loss: 0.4970 - val_accuracy: 0.9375
Epoch 4/8
5/5 [==============================] - 2s 589ms/step - loss: 0.3395 - accuracy: 0.9697 - val_loss: 0.3852 - val_accuracy: 0.8750
Epoch 5/8
5/5 [==============================] - 2s 425ms/step - loss: 0.2761 - accuracy: 0.9394 - val_loss: 0.2288 - val_accuracy: 0.9375
Epoch 6/8
5/5 [==============================] - 2s 446ms/step - loss: 0.1781 - accuracy: 0.9697 - val_loss: 0.3495 - val_accuracy: 0.8750
Epoch 7/8
5/5 [==============================] - 3s 606ms/step - loss: 0.1577 - accuracy: 0.9848 - val_loss: 0.2260 - val_accuracy: 0.9375
Epoch 8/8
5/5 [=====================

In [8]:
# The labels are stored in class_indices in dictionary form. 
# checking the labels
train_generator.class_indices

{'bokchoy': 0, 'napa': 1, 'red': 2, 'savoy': 3}

In [9]:
#training_images = 82
#validation_images = 20

#history = model.fit_generator(train_generator,
  #                 steps_per_epoch = 82,  # this should be equal to total number of images in training set. But to speed up the execution, I am only using 10000 images. Change this for better results. 
    #               epochs = 8,  # change this for better results
   #                validation_data = validation_generator)
      #             validation_steps = 20)  # this should be equal to total number of images in validation set.

In [10]:
print ('Training Accuracy = ' + str(history.history['accuracy']))
print ('Validation Accuracy = ' + str(history.history['val_accuracy']))

Training Accuracy = [0.3181818127632141, 0.6969696879386902, 0.8636363744735718, 0.9696969985961914, 0.939393937587738, 0.9696969985961914, 0.9848484992980957, 1.0]
Validation Accuracy = [0.6875, 0.75, 0.9375, 0.875, 0.9375, 0.875, 0.9375, 0.9375]
